[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainLGBmodel.ipynb)

In [0]:
!pip install lightgbm

In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [0]:
drive.mount('/content/gdrive')

In [0]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

In [0]:
s_data = 'user_train.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue'
l_col_drop = [s_col_tgt]

In [0]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

In [0]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

In [0]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



In [0]:
%%time
from sklearn.model_selection import GridSearchCV

def applyGSCV(model, param, X, Y):
    res = GridSearchCV(model, param, cv=3)
    res.fit(X, Y)
    return res


model = lgb.LGBMRegressor()

param = {
    'learning_rate': [0.08],
    'n_estimators': [300],
    'num_leaves': [64,128],
    'subsample': [0.1,0.2],
    'colsample_bytree': [0.5,0.6]
}
  
model_LGB = applyGSCV(model, param, x_train.head(10000), y_train.head(10000))



In [0]:
print(model_LGB.best_score_)
print(model_LGB.best_estimator_)


In [0]:
%%time
model_LGB.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds = 300)

In [0]:
%%time
pickle.dump(model_LGB, open(filename, 'wb'))

# files.download(filename) 